In [88]:
import spacy
from time import time
import re
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
import logging
from gensim.models.phrases import Phrases, Phraser
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import gensim

In [2]:
df2 = pd.read_csv("sentiment/complete_text.csv")
df2 = df2.loc[:,['headline', 'date', 'content']]
# parse the contents and headline, tokenise them
df2 = df2.dropna().reset_index(drop=True)
df2 = df2.drop_duplicates()
docs = []
doc_labels = []
for index, row in df2.iterrows():
    doc_labels.append(row['headline'])
    docs.append(row['headline'] + ". " + row['content'])

In [4]:
def token_clean(doc):
    text = [token.lemma_ for token in doc if not token.is_stop]
    if len(text)>2:
        text = ' '.join(text)
        return text

def preprocess(to_process):
    nlp = spacy.load("en_core_web_sm")
    docs = []
    t = time()
    index = 0
    mini_clean = (re.sub("[^A-Za-z']+", ' ', str(entry)).lower() for entry in to_process)
    clean_docs = [token_clean(doc) for doc in nlp.pipe(mini_clean, disable=["tagger", "parser", "ner"])]
    print("Time to clean up everything: {}".format((time() - t)/ 60))
    return clean_docs

In [5]:
clean_docs = preprocess(docs)

Time to clean up everything: 3.5220301230748494


In [56]:
print(len(clean_docs))
df = pd.DataFrame({'headlines': doc_labels, 'clean_docs': clean_docs})
df.shape
len(doc_labels)

37342


37342

In [57]:
df3 = df.drop_duplicates()

In [59]:
df3.shape
df3.to_csv("cleaned_docs.csv")


In [69]:
hed = df['headlines']
for i, f in hed.iteritems():
    print("yes")
    break

yes


In [70]:
class DocIterator(object):
    def __init__(self, df):
       self.labels_list = df['headlines']
       self.doc_list = df['clean_docs']
    def __iter__(self):
        for index, row in self.doc_list.iteritems():
            yield TaggedDocument(words = row.split(), tags=[self.labels_list[index]])

            

In [71]:
docit = DocIterator(df3)

In [72]:
model = Doc2Vec(vector_size=120, window=2, min_count=20, workers=4,alpha=0.025, min_alpha=0.025)

In [73]:
model.build_vocab(docit, progress_per=10000)

INFO - 09:40:49: collecting all words and their counts
INFO - 09:40:49: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 09:40:51: PROGRESS: at example #10000, processed 4060555 words (2027791/s), 54173 word types, 9267 tags
INFO - 09:40:53: PROGRESS: at example #20000, processed 8133656 words (2322880/s), 76069 word types, 18535 tags
INFO - 09:40:55: PROGRESS: at example #30000, processed 12389304 words (2263436/s), 93936 word types, 27669 tags
INFO - 09:40:56: collected 102872 word types and 33197 unique tags from a corpus of 35962 examples and 14711930 words
INFO - 09:40:56: Loading a fresh vocabulary
INFO - 09:40:56: effective_min_count=20 retains 18329 unique words (17% of original 102872, drops 84543)
INFO - 09:40:56: effective_min_count=20 leaves 14439748 word corpus (98% of original 14711930, drops 272182)
INFO - 09:40:56: deleting the raw counts dictionary of 102872 items
INFO - 09:40:56: sample=0.001 downsamples 33 most-common words
INFO - 09:40:5

In [74]:
model.train(docit, total_examples=model.corpus_count, epochs=20, report_delay=1)

INFO - 09:43:48: training model with 4 workers on 18329 vocabulary and 120 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
INFO - 09:43:49: EPOCH 1 - PROGRESS: at 2.94% examples, 340456 words/s, in_qsize 7, out_qsize 0
INFO - 09:43:50: EPOCH 1 - PROGRESS: at 5.37% examples, 366635 words/s, in_qsize 7, out_qsize 1
INFO - 09:43:51: EPOCH 1 - PROGRESS: at 8.48% examples, 376971 words/s, in_qsize 8, out_qsize 0
INFO - 09:43:52: EPOCH 1 - PROGRESS: at 11.36% examples, 382513 words/s, in_qsize 7, out_qsize 0
INFO - 09:43:53: EPOCH 1 - PROGRESS: at 14.15% examples, 384296 words/s, in_qsize 7, out_qsize 0
INFO - 09:43:54: EPOCH 1 - PROGRESS: at 15.88% examples, 366422 words/s, in_qsize 7, out_qsize 0
INFO - 09:43:55: EPOCH 1 - PROGRESS: at 17.96% examples, 346624 words/s, in_qsize 7, out_qsize 0
INFO - 09:43:56: EPOCH 1 - PROGRESS: at 20.44% examples, 336192 words/s, in_qsize 7, out_qsize 2
INFO - 09:43:57: EPOCH 1 - PROGRESS: at 23.34% examples, 341858 words/s, in_qsize 7, out_qsiz

INFO - 09:45:06: EPOCH 3 - PROGRESS: at 24.34% examples, 407279 words/s, in_qsize 7, out_qsize 0
INFO - 09:45:07: EPOCH 3 - PROGRESS: at 27.56% examples, 408945 words/s, in_qsize 7, out_qsize 0
INFO - 09:45:08: EPOCH 3 - PROGRESS: at 30.59% examples, 410294 words/s, in_qsize 7, out_qsize 0
INFO - 09:45:09: EPOCH 3 - PROGRESS: at 33.36% examples, 409694 words/s, in_qsize 7, out_qsize 0
INFO - 09:45:10: EPOCH 3 - PROGRESS: at 36.52% examples, 408860 words/s, in_qsize 7, out_qsize 0
INFO - 09:45:11: EPOCH 3 - PROGRESS: at 39.61% examples, 406458 words/s, in_qsize 8, out_qsize 0
INFO - 09:45:12: EPOCH 3 - PROGRESS: at 42.73% examples, 407345 words/s, in_qsize 7, out_qsize 0
INFO - 09:45:13: EPOCH 3 - PROGRESS: at 45.68% examples, 407327 words/s, in_qsize 7, out_qsize 0
INFO - 09:45:14: EPOCH 3 - PROGRESS: at 48.99% examples, 406860 words/s, in_qsize 8, out_qsize 0
INFO - 09:45:15: EPOCH 3 - PROGRESS: at 52.14% examples, 406713 words/s, in_qsize 7, out_qsize 0
INFO - 09:45:16: EPOCH 3 - PRO

INFO - 09:46:24: EPOCH 5 - PROGRESS: at 58.44% examples, 410974 words/s, in_qsize 7, out_qsize 0
INFO - 09:46:25: EPOCH 5 - PROGRESS: at 61.85% examples, 411527 words/s, in_qsize 7, out_qsize 0
INFO - 09:46:26: EPOCH 5 - PROGRESS: at 65.15% examples, 412672 words/s, in_qsize 8, out_qsize 0
INFO - 09:46:27: EPOCH 5 - PROGRESS: at 68.33% examples, 412992 words/s, in_qsize 7, out_qsize 0
INFO - 09:46:28: EPOCH 5 - PROGRESS: at 71.27% examples, 413486 words/s, in_qsize 7, out_qsize 0
INFO - 09:46:29: EPOCH 5 - PROGRESS: at 74.18% examples, 414276 words/s, in_qsize 7, out_qsize 0
INFO - 09:46:30: EPOCH 5 - PROGRESS: at 77.97% examples, 414592 words/s, in_qsize 7, out_qsize 0
INFO - 09:46:31: EPOCH 5 - PROGRESS: at 80.89% examples, 415177 words/s, in_qsize 6, out_qsize 1
INFO - 09:46:32: EPOCH 5 - PROGRESS: at 83.72% examples, 414659 words/s, in_qsize 8, out_qsize 0
INFO - 09:46:33: EPOCH 5 - PROGRESS: at 86.99% examples, 414920 words/s, in_qsize 8, out_qsize 0
INFO - 09:46:34: EPOCH 5 - PRO

INFO - 09:47:41: worker thread finished; awaiting finish of 1 more threads
INFO - 09:47:41: worker thread finished; awaiting finish of 0 more threads
INFO - 09:47:41: EPOCH - 7 : training on 14711930 raw words (13433072 effective words) took 32.4s, 414388 effective words/s
INFO - 09:47:42: EPOCH 8 - PROGRESS: at 3.28% examples, 399125 words/s, in_qsize 7, out_qsize 0
INFO - 09:47:43: EPOCH 8 - PROGRESS: at 6.04% examples, 416736 words/s, in_qsize 8, out_qsize 0
INFO - 09:47:44: EPOCH 8 - PROGRESS: at 9.30% examples, 418660 words/s, in_qsize 7, out_qsize 0
INFO - 09:47:45: EPOCH 8 - PROGRESS: at 12.38% examples, 420724 words/s, in_qsize 7, out_qsize 0
INFO - 09:47:46: EPOCH 8 - PROGRESS: at 15.19% examples, 423656 words/s, in_qsize 8, out_qsize 0
INFO - 09:47:47: EPOCH 8 - PROGRESS: at 18.55% examples, 421175 words/s, in_qsize 7, out_qsize 0
INFO - 09:47:48: EPOCH 8 - PROGRESS: at 21.61% examples, 419857 words/s, in_qsize 7, out_qsize 0
INFO - 09:47:49: EPOCH 8 - PROGRESS: at 25.14% exa

INFO - 09:48:58: EPOCH 10 - PROGRESS: at 34.17% examples, 417526 words/s, in_qsize 7, out_qsize 0
INFO - 09:48:59: EPOCH 10 - PROGRESS: at 37.35% examples, 416762 words/s, in_qsize 7, out_qsize 0
INFO - 09:49:00: EPOCH 10 - PROGRESS: at 40.68% examples, 416245 words/s, in_qsize 7, out_qsize 0
INFO - 09:49:01: EPOCH 10 - PROGRESS: at 43.75% examples, 417870 words/s, in_qsize 8, out_qsize 0
INFO - 09:49:02: EPOCH 10 - PROGRESS: at 47.02% examples, 417432 words/s, in_qsize 8, out_qsize 0
INFO - 09:49:03: EPOCH 10 - PROGRESS: at 50.43% examples, 417151 words/s, in_qsize 8, out_qsize 0
INFO - 09:49:04: EPOCH 10 - PROGRESS: at 53.85% examples, 418643 words/s, in_qsize 7, out_qsize 0
INFO - 09:49:05: EPOCH 10 - PROGRESS: at 56.26% examples, 419685 words/s, in_qsize 7, out_qsize 0
INFO - 09:49:06: EPOCH 10 - PROGRESS: at 59.67% examples, 418692 words/s, in_qsize 7, out_qsize 0
INFO - 09:49:07: EPOCH 10 - PROGRESS: at 62.89% examples, 418060 words/s, in_qsize 7, out_qsize 0
INFO - 09:49:08: EPO

INFO - 09:50:15: EPOCH 12 - PROGRESS: at 60.52% examples, 382825 words/s, in_qsize 7, out_qsize 0
INFO - 09:50:16: EPOCH 12 - PROGRESS: at 63.87% examples, 382925 words/s, in_qsize 7, out_qsize 0
INFO - 09:50:17: EPOCH 12 - PROGRESS: at 66.50% examples, 384055 words/s, in_qsize 7, out_qsize 0
INFO - 09:50:18: EPOCH 12 - PROGRESS: at 69.28% examples, 383869 words/s, in_qsize 8, out_qsize 0
INFO - 09:50:19: EPOCH 12 - PROGRESS: at 71.95% examples, 384431 words/s, in_qsize 7, out_qsize 0
INFO - 09:50:20: EPOCH 12 - PROGRESS: at 74.65% examples, 384851 words/s, in_qsize 7, out_qsize 0
INFO - 09:50:21: EPOCH 12 - PROGRESS: at 78.02% examples, 384415 words/s, in_qsize 7, out_qsize 0
INFO - 09:50:22: EPOCH 12 - PROGRESS: at 80.71% examples, 384516 words/s, in_qsize 7, out_qsize 0
INFO - 09:50:23: EPOCH 12 - PROGRESS: at 83.30% examples, 384501 words/s, in_qsize 7, out_qsize 0
INFO - 09:50:24: EPOCH 12 - PROGRESS: at 86.16% examples, 384484 words/s, in_qsize 8, out_qsize 0
INFO - 09:50:25: EPO

INFO - 09:51:32: EPOCH 14 - PROGRESS: at 68.35% examples, 349983 words/s, in_qsize 8, out_qsize 0
INFO - 09:51:33: EPOCH 14 - PROGRESS: at 70.87% examples, 350940 words/s, in_qsize 7, out_qsize 0
INFO - 09:51:34: EPOCH 14 - PROGRESS: at 73.12% examples, 350033 words/s, in_qsize 7, out_qsize 0
INFO - 09:51:35: EPOCH 14 - PROGRESS: at 74.68% examples, 344216 words/s, in_qsize 7, out_qsize 0
INFO - 09:51:36: EPOCH 14 - PROGRESS: at 75.97% examples, 337462 words/s, in_qsize 8, out_qsize 0
INFO - 09:51:37: EPOCH 14 - PROGRESS: at 78.70% examples, 337879 words/s, in_qsize 7, out_qsize 0
INFO - 09:51:38: EPOCH 14 - PROGRESS: at 81.00% examples, 339759 words/s, in_qsize 7, out_qsize 0
INFO - 09:51:39: EPOCH 14 - PROGRESS: at 83.81% examples, 339572 words/s, in_qsize 8, out_qsize 0
INFO - 09:51:40: EPOCH 14 - PROGRESS: at 86.07% examples, 338586 words/s, in_qsize 7, out_qsize 0
INFO - 09:51:41: EPOCH 14 - PROGRESS: at 89.05% examples, 339084 words/s, in_qsize 7, out_qsize 0
INFO - 09:51:42: EPO

INFO - 09:52:51: EPOCH 16 - PROGRESS: at 54.77% examples, 285175 words/s, in_qsize 7, out_qsize 0
INFO - 09:52:52: EPOCH 16 - PROGRESS: at 56.21% examples, 284245 words/s, in_qsize 7, out_qsize 0
INFO - 09:52:53: EPOCH 16 - PROGRESS: at 57.86% examples, 283256 words/s, in_qsize 7, out_qsize 0
INFO - 09:52:54: EPOCH 16 - PROGRESS: at 61.10% examples, 286447 words/s, in_qsize 7, out_qsize 0
INFO - 09:52:55: EPOCH 16 - PROGRESS: at 64.20% examples, 288977 words/s, in_qsize 8, out_qsize 0
INFO - 09:52:56: EPOCH 16 - PROGRESS: at 66.57% examples, 292070 words/s, in_qsize 7, out_qsize 0
INFO - 09:52:57: EPOCH 16 - PROGRESS: at 69.17% examples, 294187 words/s, in_qsize 8, out_qsize 0
INFO - 09:52:58: EPOCH 16 - PROGRESS: at 71.38% examples, 294540 words/s, in_qsize 7, out_qsize 0
INFO - 09:52:59: EPOCH 16 - PROGRESS: at 73.89% examples, 297320 words/s, in_qsize 7, out_qsize 0
INFO - 09:53:00: EPOCH 16 - PROGRESS: at 77.07% examples, 299283 words/s, in_qsize 8, out_qsize 0
INFO - 09:53:01: EPO

INFO - 09:54:09: EPOCH 18 - PROGRESS: at 38.70% examples, 366761 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:10: EPOCH 18 - PROGRESS: at 41.16% examples, 365765 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:11: EPOCH 18 - PROGRESS: at 44.02% examples, 367238 words/s, in_qsize 8, out_qsize 0
INFO - 09:54:12: EPOCH 18 - PROGRESS: at 47.11% examples, 367962 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:13: EPOCH 18 - PROGRESS: at 49.39% examples, 362684 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:14: EPOCH 18 - PROGRESS: at 51.85% examples, 360167 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:15: EPOCH 18 - PROGRESS: at 53.81% examples, 354942 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:16: EPOCH 18 - PROGRESS: at 55.96% examples, 353217 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:17: EPOCH 18 - PROGRESS: at 56.94% examples, 346543 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:18: EPOCH 18 - PROGRESS: at 58.14% examples, 337018 words/s, in_qsize 7, out_qsize 0
INFO - 09:54:19: EPO

INFO - 09:55:27: EPOCH 20 - PROGRESS: at 33.05% examples, 314213 words/s, in_qsize 8, out_qsize 1
INFO - 09:55:28: EPOCH 20 - PROGRESS: at 34.38% examples, 306684 words/s, in_qsize 7, out_qsize 0
INFO - 09:55:29: EPOCH 20 - PROGRESS: at 35.80% examples, 297800 words/s, in_qsize 7, out_qsize 0
INFO - 09:55:30: EPOCH 20 - PROGRESS: at 37.98% examples, 295910 words/s, in_qsize 7, out_qsize 0
INFO - 09:55:31: EPOCH 20 - PROGRESS: at 39.46% examples, 290403 words/s, in_qsize 7, out_qsize 0
INFO - 09:55:32: EPOCH 20 - PROGRESS: at 41.57% examples, 289820 words/s, in_qsize 7, out_qsize 0
INFO - 09:55:33: EPOCH 20 - PROGRESS: at 43.66% examples, 290092 words/s, in_qsize 8, out_qsize 0
INFO - 09:55:34: EPOCH 20 - PROGRESS: at 46.11% examples, 291233 words/s, in_qsize 7, out_qsize 0
INFO - 09:55:35: EPOCH 20 - PROGRESS: at 49.04% examples, 292236 words/s, in_qsize 7, out_qsize 0
INFO - 09:55:36: EPOCH 20 - PROGRESS: at 51.24% examples, 291277 words/s, in_qsize 7, out_qsize 0
INFO - 09:55:37: EPO

In [75]:
model.save("doc2vec.model")

INFO - 10:01:45: saving Doc2Vec object under doc2vec.model, separately None
INFO - 10:01:46: saved doc2vec.model


In [35]:
label = doc_labels[245]

In [27]:
print(label)

Chamber chief sets out five principles to survive Brexit


In [85]:
index = 4500
headlines = [doc_labels[index]]
for item in model.docvecs.most_similar(index):
    (label, _) = item
    headlines.append(label)
    
    

In [89]:
word2vec = gensim.models.Word2Vec.load("word2vec.model")

INFO - 10:27:34: loading Word2Vec object from word2vec.model
INFO - 10:27:34: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 10:27:34: setting ignored attribute vectors_norm to None
INFO - 10:27:34: loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
INFO - 10:27:34: loading trainables recursively from word2vec.model.trainables.* with mmap=None
INFO - 10:27:34: setting ignored attribute cum_table to None
INFO - 10:27:34: loaded word2vec.model


In [94]:
df = pd.read_csv("save.csv")

MemoryError: Unable to allocate 847. MiB for an array with shape (10534, 10535) and data type float64

In [92]:
df.columns

Index(['Unnamed: 0', 'clean_lines'], dtype='object')